### Consolidation simulation

This notebook models consolidation as teacher-student learning, in which initial representations of memories are replayed to train a generative model.

#### End-to-end simulation example

For the Shapes3D dataset, the following code:
* Encodes 10000 images (i.e. memories) in the modern Hopfield network.
* Gives the Hopfield network random noise as an input, and gets the outputs (which should be the stored memories). The outputs of this stage are saved in the mhn_memories folder for re-use; to regenerate from scratch delete the contents of the folder.
* Trains a variational autoencoder on the 'memories', and tests its recall.
* Generates plots of semantic decoding accuracy and reconstruction error over the course of training.
* Displays the latent space (projected into 2D and colour coded by class).
* Runs a set of other tests, e.g. of interpolation between items.
* Saves the outputs to a PDF in the 'outputs' folder, together with the loss history.
* Examples of recall in the extended version of the model are saved in the 'hybrid_model' folder.

#### Local installation:

Tested with tensorflow 2.11.0 and Python 3.10.9.

In [ ]:
!pip install -r requirements.txt --upgrade

#### Colab installation:

In [ ]:
!git clone https://github.com/ellie-as/generative-memory.git
%cd /content/generative-memory/
!pip install -r requirements.txt

In [ ]:
%cd /content/generative-memory

import sys
sys.path.append('/content/generative-memory')

#### Results:

In [ ]:
from end_to_end import *
from extended_model import *
import tensorflow as tf
import numpy as np

import warnings
warnings.filterwarnings("ignore")

# set tensorflow and numpy random seeds to make outputs reproducible
tf.random.set_seed(123)
np.random.seed(123)

# set tensorflow image format to arrays of dim (width, height, num_channels)
tf.keras.backend.set_image_data_format('channels_last')

# set number of epochs (actually the max number as early stopping is enabled)
generative_epochs = 50
# set number of images
num_ims = 10000
# set encoding thresholds in extended model
threshold_low = 0.03
threshold_high = 0.15

# dataset specific parameters:
# a lower value of beta is set for the larger image datasets to avoid overflow errors
params = {
    'hopfield_beta': 5,
    'latent_dim': 20
}

dataset = 'shapes3d'

# run 'basic' version of model with parameters specified for dataset
net, vae = run_end_to_end(dataset=dataset, 
                          generative_epochs=generative_epochs, 
                          num=num_ims, 
                          latent_dim=params['latent_dim'], 
                          interpolate=True,
                          do_vector_arithmetic=True,
                          kl_weighting=1,
                          lr = 0.001,
                          hopfield_beta=params['hopfield_beta'],
                          use_weights=True)
# plot the stages of recall in the 'extended' model, with the lower error threshold
test_extended_model(dataset=dataset, 
                vae=vae, 
                latent_dim=params['latent_dim'],
                threshold = threshold_low)
# plot the stages of recall in the 'extended' model, with the higher error threshold
test_extended_model(dataset=dataset, 
                vae=vae, 
                latent_dim=params['latent_dim'],
                threshold = threshold_high)